Dado que el entrenamiento de redes neuronales es una tarea  muy costosa, **se recomienda ejecutar el notebooks en [Google Colab](https://colab.research.google.com)**, por supuesto también se puede ejecutar en local.

Al entrar en [Google Colab](https://colab.research.google.com) bastará con hacer click en `upload` y subir este notebook. No olvide luego descargarlo en `File->Download .ipynb`

**El examen deberá ser entregado con las celdas ejecutadas, si alguna celda no está ejecutadas no se contará.**

El examen se divide en tres partes, con la puntuación que se indica a continuación. La puntuación máxima será 10.

    
- [Actividad 1: Redes Recurrentes](#actividad_1): 10 pts
    - [Cuestión 1](#3.1): 2.5 pt
    - [Cuestión 2](#3.2): 2.5 pt
    - [Cuestión 3](#3.3): 2.5 pts
    - [Cuestión 4](#3.4): 1.25 pts
    - [Cuestión 5](#3.5): 1.25 pts



In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

<a name='actividad_1'></a>
# Actividad 1: Redes Recurrentes


- [Cuestión 1](#3.1): 2.5 pt
- [Cuestión 2](#3.2): 2.5 pt
- [Cuestión 3](#3.3): 2.5 pts
- [Cuestión 4](#3.4): 1.25 pts
- [Cuestión 5](#3.5): 1.25 pts

Vamos a usar un dataset de las temperaturas mínimas diarias en Melbourne. La tarea será la de predecir la temperatura mínima en dos días. Puedes usar técnicas de series temporales vistas en otras asignaturas, pero no es necesario.


In [2]:
dataset_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv'
data_dir = tf.keras.utils.get_file('daily-min-temperatures.csv', origin=dataset_url)

In [3]:
df = pd.read_csv(data_dir, parse_dates=['Date'])
df.head()

,Date,Temp
0,1981-01-01,20.7
1,1981-01-02,17.9
2,1981-01-03,18.8
3,1981-01-04,14.6
4,1981-01-05,15.8


In [4]:
temperatures = df['Temp'].values
print('number of samples:', len(temperatures))
train_data = temperatures[:3000]
test_data = temperatures[3000:]
print('number of train samples:', len(train_data))
print('number of test samples:', len(test_data))
print('firsts trainn samples:', train_data[:10])

number of samples: 3650
number of train samples: 3000
number of test samples: 650
firsts trainn samples: [20.7 17.9 18.8 14.6 15.8 15.8 15.8 17.4 21.8 20. ]


<a name='3.1'></a>
## Cuestión 1: Convierta `train_data` y `test_data`  en ventanas de tamaño 5, para predecir el valor en 2 días

En la nomenclatura de [Introduction_to_RNN_Time_Series.ipynb](https://github.com/ezponda/intro_deep_learning/blob/main/class/RNN/Introduction_to_RNN_Time_Series.ipynb)
```python
past, future = (5, 2)
```

Para las primeras 10 muestras de train_data `[20.7, 17.9, 18.8, 14.6, 15.8, 15.8, 15.8, 17.4, 21.8, 20. ]` el resultado debería ser:

```python
x[0] : [20.7, 17.9, 18.8, 14.6, 15.8] , y[0]: 15.8
x[1] : [17.9, 18.8, 14.6, 15.8, 15.8] , y[1]: 17.4
x[2] : [18.8, 14.6, 15.8, 15.8, 15.8] , y[2]: 21.8
x[3] : [14.6, 15.8, 15.8, 15.8, 17.4] , y[3]: 20.             
```

In [5]:
# windowing function

def create_windows(data, window_size, horizon, target_col_idx, shuffle=False):
    """
    Creates a dataset from the given time series data using NumPy.

    Parameters:
    data (np.ndarray or pd.DataFrame): Time series data with multiple features.
    window_size (int): The number of past time steps to use as input features.
    horizon (int): The number of future time steps to predict.
    target_col_idx (int): The index of the target column in the input data.
    shuffle (bool): Whether to shuffle the data or not.

    Returns:
    tuple: A tuple containing the input-output pairs (X, y) as NumPy arrays.
    """
    if isinstance(data, pd.DataFrame):
        data = data.values

    X, y = [], []
    for i in range(len(data) - window_size - horizon + 1):
        X.append(data[i:i+window_size, :])
        y.append(data[i+window_size+horizon-1, target_col_idx])

    X, y = np.array(X), np.array(y)

    if shuffle:
        indices = np.arange(X.shape[0])
        np.random.shuffle(indices)
        X, y = X[indices], y[indices]

    return X, y

In [6]:
past, future = (5, 2)
X_train, y_train = create_windows(pd.DataFrame(train_data, columns=["Temp"]), past, future, 0)
X_test, y_test = create_windows(pd.DataFrame(test_data, columns=["Temp"]), past, future, 0)

<a name='3.2'></a>
## Cuestión 2: Cree un modelo recurrente de dos capas GRU para predecir con las ventanas de la cuestión anterior.


In [7]:
inputs = keras.layers.Input(shape=(5, 1))
                           
x = layers.GRU(32, return_sequences=True)(inputs)
x1 = layers.GRU(16)(x)
outputs = layers.Dense(1)(x1)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer="adam", loss="mse")
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 5, 1)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 5, 32)          │         3,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 16)             │         2,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,777 (22.57 KB)

 Trainable params: 5,777 (22.57 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
es_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0, patience=10)

history = model.fit(
    X_train, y_train,
    epochs=200,
    validation_split=0.2, shuffle=True, batch_size = 64, callbacks=[es_callback]
)

Epoch 1/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 97.9934 - val_loss: 75.9569
Epoch 2/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 55.0074 - val_loss: 59.0115
Epoch 3/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 43.1714 - val_loss: 50.6280
Epoch 4/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 37.5170 - val_loss: 44.0979
Epoch 5/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 31.9879 - val_loss: 38.7817
Epoch 6/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 28.1942 - val_loss: 34.3549
Epoch 7/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 26.6841 - val_loss: 30.7567
Epoch 8/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 23.2887 - val_loss: 27.8530
Epoch 9/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 20.3746 - val_loss: 25.5100
Epoch 10/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 19.4857 - val_loss: 23.5982
Epoch 11/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 18.7570 - val_loss: 22.0906
Epoch 12/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 

In [9]:
results = model.evaluate(X_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.8698 
Test Loss: 7.008089065551758


<a name='3.3'></a>
## Cuestión 3: Añada más features a la series temporal, por ejemplo `portion_year`. Cree un modelo que mejore al anterior.


In [10]:
## Puede añadir más features
df['portion_year'] = df['Date'].dt.dayofyear / 365.0
df_multi = df[['Temp', 'portion_year']].copy()

## train - test split
train_data = df_multi.iloc[:3000].copy()
test_data = df_multi.loc[3000:, :].copy()

In [11]:
## Create windows
X_train, y_train = create_windows(train_data, past, future, target_col_idx=0)
X_test, y_test = create_windows(test_data, past, future, target_col_idx=0)

In [12]:
inputs = keras.layers.Input(shape=(5, 2))

x = layers.LSTM(64, return_sequences=True)(inputs)
x1 = layers.LSTM(32, return_sequences=True)(x)
x2 = layers.LSTM(16)(x1)
outputs = layers.Dense(1)(x2)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer="adam", loss="mse")
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 5, 2)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 5, 64)          │        17,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 5, 32)          │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 16)             │         3,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,721 (127.82 KB)

 Trainable params: 32,721 (127.82 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
es_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0, patience=10)

history = model.fit(
    X_train, y_train,
    epochs=200,
    validation_split=0.2, shuffle=True, batch_size = 64, callbacks=[es_callback]
)

Epoch 1/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - loss: 121.2001 - val_loss: 93.8161
Epoch 2/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 66.0423 - val_loss: 66.8937
Epoch 3/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 48.1305 - val_loss: 55.5649
Epoch 4/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 39.5169 - val_loss: 47.6521
Epoch 5/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 36.3183 - val_loss: 41.5188
Epoch 6/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 30.5598 - val_loss: 36.4467
Epoch 7/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 27.0599 - val_loss: 32.3491
Epoch 8/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 23.4781 - val_loss: 29.1222
Epoch 9/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 21.5177 - val_loss: 26.4576
Epoch 10/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 20.9785 - val_loss: 24.3979
Epoch 11/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 19.1769 - val_loss: 22.6946
Epoch 12/200
38/38 ━━━━━━━━━━━━━━━━━━━━

In [14]:
results = model.evaluate(X_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.7280
Test Loss: 6.4251298904418945


<a name='3.4'></a>
## Cuestión 4: ¿En cuáles de estas aplicaciones se usaría un arquitectura 'many-to-one'?

**a)** Clasificación de sentimiento en textos

**b)** Verificación de voz para iniciar el ordenador.

**c)** Generación de música.

**d)** Un clasificador que clasifique piezas de música según su autor.


Se usaría en las aplicaciones **a)**, **b)** y **d)**.

<a name='3.5'></a>
## Cuestión 5: ¿Qué ventajas aporta el uso de word embeddings?

**a)** Permiten reducir la dimensión de entrada respecto al one-hot encoding.

**b)** Permiten descubrir la similaridad entre palabras de manera más intuitiva que con one-hot encoding.

**c)** Son una manera de realizar transfer learning en nlp.

**d)** Permiten visualizar las relaciones entre palabras con métodos de reducción de dimensioones como el PCA.


Las ventajas del uso de *word embeddings* son: **a)**, **b)**, **c)** y **d)**